# Python Tip 5

author@snowhyzhang

- [对list进行滑动取值](#move_fetch)
- [对多个list做笛卡尔积](#cartesian)
- [list的copy和deepcopy](#list_copy)
- [dict与lambda表达式相结合](#dict_lambda)

---
<a name='move_fetch'>

## 对list进行滑动取值

在序列分析中，设定一个窗口大小，滑动从`list`中的取值是常有操作，我们可以利用`zip`函数返回匹配的最短长度的特性，方便的从`list`中进行滑动取值

In [1]:
def move_fetch(l, windows_size):
    windows = [l[i:] for i in range(windows_size)]
    return zip(*windows)

l = [1, 2, 3, 4, 5, 6]
move_list = list(move_fetch(l, 3))
print(move_list)
print('move avg:', [sum(x) / len(x) for x in move_list])

[(1, 2, 3), (2, 3, 4), (3, 4, 5), (4, 5, 6)]
move avg: [2.0, 3.0, 4.0, 5.0]


---
<a name='cartesian'>

## 对多个list做笛卡尔积

可以使用嵌套的列推导方式对两个`list`做笛卡尔积，例如

In [2]:
l1 = [0, 1, 2]
l2 = [10, 11]

[(x, y) for x in l1 for y in l2]

[(0, 10), (0, 11), (1, 10), (1, 11), (2, 10), (2, 11)]

但如果有更多`list`一起做笛卡尔积的话，列推导方式的代码可读性会比较差，我们可以使用`itertools`模块中的`product`，可以方便的完成这个笛卡尔积

In [3]:
from itertools import product

l1 = [0, 1, 2]
l2 = [10, 11]
l3 = [100, 200]

for p in product(l1, l2, l3):
    print(p, end=' ')

(0, 10, 100) (0, 10, 200) (0, 11, 100) (0, 11, 200) (1, 10, 100) (1, 10, 200) (1, 11, 100) (1, 11, 200) (2, 10, 100) (2, 10, 200) (2, 11, 100) (2, 11, 200) 

In [4]:
['{}-{}-{}'.format(*p) for p in product(l1, l2, l3)]

['0-10-100',
 '0-10-200',
 '0-11-100',
 '0-11-200',
 '1-10-100',
 '1-10-200',
 '1-11-100',
 '1-11-200',
 '2-10-100',
 '2-10-200',
 '2-11-100',
 '2-11-200']

---
<a name='list_copy'>

## list的copy和deepcopy

`list`直接使用`=`来“复制”时，当修改其中一个`list`中的元素，“另一个”`list`也会受到影响

In [5]:
l1 = [1, 2, 3, 4, 5]
l2 = l1
l1[0] = 100
l2

[100, 2, 3, 4, 5]

这是由于变量`l1`和`l2`指向的是同一个`list`对象，因此，实际上对`list`的修改的是同一个`list`  
我们可以调用`list`的`copy`方法，这样会对`list`进行一个复制

In [6]:
l1 = [1, 2, 3, 4, 5]
l2 = l1.copy()
l1[0] = 100
l2

[1, 2, 3, 4, 5]

但是如果是嵌套`list`，那么只调用`copy`方法是不够的

In [7]:
l1 = [[1, 2, 3], [4, 5, 6]]
l2 = l1.copy()
l1[0][0] = 100
l2

[[100, 2, 3], [4, 5, 6]]

这是由于`copy`方法只复制了第一层数据，如果是嵌套的话，那么里层指向的还是同一个`list`，这里我们需要`copy`模块中的`deepcopy`方法

In [8]:
from copy import deepcopy
l1 = [[1, 2, 3], [4, 5, 6]]
l2 = deepcopy(l1)
l1[0][0] = 100
l2

[[1, 2, 3], [4, 5, 6]]

`deepcopy`会完全将`list`复制下来，因此数据比较大的时候，非常占用内存，需要小心使用

---
<a name='dict_lambda'>

## dict与lambda表达式相结合

有时候我们需要对不同的关键字做出不同的操作，如关键字为`increase`时，我们对输入数字加1；关键字为`decrease`，我们对输入数字减1，我们可以使用`if-else`结构来实现

In [9]:
def my_op(op, x, y=None):
    if op == 'increase':
        return x + 1
    elif op == 'decrease':
        return x - 1
    elif op == 'update':
        return x + y + 1
    elif op == 'swap':
        return y, x
    else:
        print('No such op!')
        return None

但当有太多可判断的条件时，大量`if-else`结构并不利于代码的维护，并且无法进行动态的修改。这时我们可以使用`dict`与`lambda`表达式相结合的方式来实现这样的操作，这样可以动态的对`dict`做出改变

In [10]:
op_switch = {
    'increase': lambda x: x + 1,
    'decrease': lambda x: x - 1,
    'update': lambda x, y: x + y + 1,
    'swap': lambda x, y: (y, x)
}

def my_op(op, *args):
    # 新增/修改op
    if op == 'add_op':
        # 如果op是字符串形式，那么使用eval函数对字符串求值
        if isinstance(args[1], str):
            op = eval(args[1])
        else:
            op = args[1]
        op_switch[args[0]] = op
        print('added op...')
        return None
    # 判断op是否合法
    if op not in op_switch.keys():
        print('No such op!')
        return None
    return op_switch[op](*args)

In [11]:
print('increase: ', my_op('increase', 1))
print('decrease: ', my_op('decrease', 1))
print('update: ', my_op('update', 1, 2))
print('swap: ', my_op('swap', 1, 2))

increase:  2
decrease:  0
update:  4
swap:  (2, 1)


删除`swap`

In [12]:
del op_switch['swap']
my_op('swap', 1, 2)

No such op!


新增`op`，这里lambda表达式使用的是字符串形式

In [13]:
# 设定新的操作
my_new_op = ('relu', 'lambda x: max(0, x)')
my_op('add_op', *my_new_op)
my_op('relu', -10)

added op...


0